In [20]:
#Importar la señal
%matplotlib notebook

import matplotlib.pyplot as plt
import sys
from tds_utils import *
import numpy as np
from IPython.display import Audio
import math

import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import scipy.signal as sig
import sounddevice as sd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split





import scipy.io.wavfile as wf


class clasePalabra:

    def __init__(self):

        self.fs = 0
        self.y = 0
        self.NumVentana = 0
        self.MuestraVentana = 0
        
#i=1
#arrayPalabras = []
nombrePersonas = ["Patri","Adri","Isa","Filippo","Miguelangel"]
arrayPersonas =[]
for j in range(0,len(nombrePersonas)):
    arrayPalabras = []
    i=1
    while i < 11:
        filename = nombrePersonas[j]+"/"+str(i)+".wav"
        fs,y = wf.read(filename)
        palabra  = clasePalabra()
        palabra.fs= fs
        palabra.y = y
        t = np.arange(0,len(palabra.y)/palabra.fs,1/palabra.fs)
        palabra.numVentana =  math.floor(max(t)/0.02)
        palabra.muestrasVentana = math.floor(len(palabra.y)/palabra.numVentana)
        arrayPalabras.append(palabra)
        i=i+1
        
    arrayPersonas.append(arrayPalabras)
    
    #Play the signal at fs
    #print(arrayPersonas)
    print(fs)
    #print(arrayPersonas)
    sd.play(palabra.y,palabra.fs)

44100
44100
44100
44100
44100


In [12]:

class claseLocutor:

    def __init__(self):
        self.nombre = ""
        self.mediaPitch = 0
        self.mediaLpc = []

        
#arrayMedias = [] 
#arrayLpc = []
arrayLocutor = []
for x in range (0,len(arrayPersonas)):
    #print(len(arrayPersonas[x]))
    arrayMedias = []
    arrayLpc = []

    for z in range (0,len(arrayPersonas[x])):
        i = 0
        pitch = []
        mediaTotal = 0
        k = []
        while i < arrayPersonas[x][z].numVentana:
            seg = arrayPersonas[x][z].y[arrayPersonas[x][z].muestrasVentana*(i):arrayPersonas[x][z].muestrasVentana*(i+1)]
            Canal_1 = seg[:,0]
            psd1,f1 = my_spectra(Canal_1,fs)
            length = 0
            while f1[length]<0:
                length = length + 1
            psd=psd1[length:]
            f=f1[length:]
            umbral = 50
            umbralBaja = 0
            j=3
            l=True
            lpc_f=False
            p0=13
            while l==True:
                if psd[j]>umbral:
                    lpc_f=True
                if psd[j+1]<psd[j]:
                    j=j+1
                else:
                    l=False
            while j<23:
                if psd[j]>umbral:
                    lpc_f=True
                if psd[j+2] < psd[j] and psd[j+2] > umbral :
                    pitch.append(f[j])
                    k.append(i)
                    break
                j=j+1
                #pitch=0.0
            if lpc_f==True:
                h = sig.hamming(len(Canal_1)) #hamming window
                arrayLpc.append (predlin2(Canal_1,p0,h))
            #print(arrayLpc)
            i = i+1
        #print(pitch)
        #print(k)
        media=0
        media = np.mean(pitch[0:len(pitch)])
        arrayMedias.append(media)
        
    mediaTotal = np.mean(arrayMedias[0:len(arrayMedias)])
    print (mediaTotal)
    mediaLpc = [0,0,0,0,0,0,0,0,0,0,0,0,0]
    for z in range (0,len(arrayLpc)):
        mediaLpc=mediaLpc + arrayLpc[z]
    mediaLpc= mediaLpc/len(arrayLpc)
#---------------------------------------------------------------
    locutor = claseLocutor()
    locutor.nombre = nombrePersonas[x]
    locutor.mediaPitch = mediaTotal
    locutor.mediaLpc = mediaLpc
    print(locutor.nombre)
    arrayLocutor.append(locutor)
    print(mediaLpc)


197.83901854916084
Patri
[-1.67905435  0.78364727 -0.18337082  0.05510457  0.05228467 -0.09745915
  0.09303902 -0.0090024  -0.07830035  0.03323202  0.02749578 -0.02227957
  0.02825137]
133.36701769890433
Adri
[-1.85001799  1.06185328 -0.16987319 -0.07995748 -0.01096773 -0.00405746
  0.12831394 -0.19184155  0.07518871  0.1539532  -0.15047158  0.03357181
  0.01000688]
180.44712967218138
Isa
[-1.57237252  0.59701634 -0.06013261 -0.253945    0.31978515 -0.03441622
 -0.04164033  0.0857804  -0.01116903  0.01730025 -0.02436626 -0.11523719
  0.09791203]
117.38220405356046
Filippo
[-1.47359969e+00  3.46615203e-01  3.30910437e-01 -3.50475777e-01
  1.57233587e-02  1.91551531e-01  2.31271586e-02 -1.37265308e-01
 -4.71999309e-02  6.32337841e-02 -1.17732889e-03  2.08467510e-02
  2.12328865e-02]
98.21078364605505
Miguelangel
[-1.27861544 -0.31589295  0.68945335 -0.04692199 -0.17978594  0.19067422
  0.02059574 -0.19977964 -0.00688762  0.15730546  0.03734375 -0.05586574
 -0.00902037]


In [18]:
#En esta función se tendrá que organizar los datos en una matriz
#Primero se va a meter lo datos en un único array ara después transformarlo en una matriz
locutorTest = []
for i in range (0,len(arrayLocutor)):
    locutorTest.append(arrayLocutor[i].mediaPitch)
    for j in range (0,len(arrayLocutor[i].mediaLpc)):
        locutorTest.append(arrayLocutor[i].mediaLpc[j])
            
matrizLocutorTest = np.array(locutorTest).reshape(5,14)
vectorLocutorTest = np.array(nombrePersonas).reshape(1,5)

print(matrizLocutorTest)
print(vectorLocutorTest)
# Creamos el modelo de NB

model_NB = GaussianNB()

#entrenamos
X_train, X_test, y_train, y_test = train_test_split(matrizLocutorTest,nombrePersonas, test_size=2)

model_NB.fit(X_train,y_train)

print(model_NB.sigma_)
y_hat_NB = model_NB.predict(X_test)
print(y_hat_NB)

#Utilizaremos y_test para comparar la estimacion con lo que realmente sabemos que hay

#Calculamos el accuracy
temp = y_hat_NB == y_test
print(temp)

acc = np.mean(y_hat_NB == y_test)
print(acc)

#Cálculos para 

[[ 1.97839019e+02 -1.67905435e+00  7.83647271e-01 -1.83370822e-01
   5.51045665e-02  5.22846684e-02 -9.74591496e-02  9.30390232e-02
  -9.00240107e-03 -7.83003453e-02  3.32320164e-02  2.74957790e-02
  -2.22795721e-02  2.82513746e-02]
 [ 1.33367018e+02 -1.85001799e+00  1.06185328e+00 -1.69873186e-01
  -7.99574801e-02 -1.09677322e-02 -4.05746063e-03  1.28313939e-01
  -1.91841553e-01  7.51887085e-02  1.53953204e-01 -1.50471577e-01
   3.35718054e-02  1.00068843e-02]
 [ 1.80447130e+02 -1.57237252e+00  5.97016343e-01 -6.01326087e-02
  -2.53944997e-01  3.19785155e-01 -3.44162175e-02 -4.16403264e-02
   8.57804016e-02 -1.11690333e-02  1.73002514e-02 -2.43662647e-02
  -1.15237187e-01  9.79120275e-02]
 [ 1.17382204e+02 -1.47359969e+00  3.46615203e-01  3.30910437e-01
  -3.50475777e-01  1.57233587e-02  1.91551531e-01  2.31271586e-02
  -1.37265308e-01 -4.71999309e-02  6.32337841e-02 -1.17732889e-03
   2.08467510e-02  2.12328865e-02]
 [ 9.82107836e+01 -1.27861544e+00 -3.15892954e-01  6.89453351e-01
  

In [22]:
class claseLocutor:

    def __init__(self):
        self.nombre = ""
        self.mediaPitch = 0
        self.mediaLpc = []

        
#arrayMedias = [] 
#arrayLpc = []
arrayPalabras = []
arrayLocutores = []
for x in range (0,len(arrayPersonas)):
    #print(len(arrayPersonas[x]))
    arrayMedias = []
    arrayLpc = []
    for z in range (0,len(arrayPersonas[x])):
        array = []
        i = 0
        pitch = []
        mediaTotal = 0
        k = []
        while i < arrayPersonas[x][z].numVentana:
            seg = arrayPersonas[x][z].y[arrayPersonas[x][z].muestrasVentana*(i):arrayPersonas[x][z].muestrasVentana*(i+1)]
            Canal_1 = seg[:,0]
            psd1,f1 = my_spectra(Canal_1,fs)
            length = 0
            while f1[length]<0:
                length = length + 1
            psd=psd1[length:]
            f=f1[length:]
            umbral = 50
            umbralBaja = 0
            j=3
            l=True
            lpc_f=False
            p0=13
            while l==True:
                if psd[j]>umbral:
                    lpc_f=True
                if psd[j+1]<psd[j]:
                    j=j+1
                else:
                    l=False
            while j<23:
                if psd[j]>umbral:
                    lpc_f=True
                if psd[j+2] < psd[j] and psd[j+2] > umbral :
                    pitch.append(f[j])
                    k.append(i)
                    break
                j=j+1
                #pitch=0.0
            if lpc_f==True:
                h = sig.hamming(len(Canal_1)) #hamming window
                arrayLpc.append (predlin2(Canal_1,p0,h))
            #print(arrayLpc)
            i = i+1
        #print(pitch)
        #print(k)
        media=0
        media = np.mean(pitch[0:len(pitch)])
        array.append(media)
        mediaLpc = [0,0,0,0,0,0,0,0,0,0,0,0,0]
        for z in range (0,len(arrayLpc)):
            mediaLpc=mediaLpc + arrayLpc[z]
        mediaLpc= mediaLpc/len(arrayLpc)
        for p in range (0,p0):
            array.append(mediaLpc[p])
        arrayPalabras.append(array)
        arrayLocutores.append(nombrePersonas[x])
    #print(arrayMedias)   
    mediaTotal = np.mean(arrayMedias[0:len(arrayMedias)])
    #print (mediaTotal)
    
    
#---------------------------------------------------------------
    locutor = claseLocutor()
    locutor.nombre = nombrePersonas[x]
    locutor.mediaPitch = mediaTotal
    locutor.mediaLpc = mediaLpc
    #print(locutor.nombre)
    #arrayLocutores.append(locutor)
    #print(mediaLpc)
#print(arrayLocutores)
#print(arrayPalabras)

# Creamos el modelo de NB

model_NB = GaussianNB()

#entrenamos
for i in range(0,10):
    X_train, X_test, y_train, y_test = train_test_split(arrayPalabras,arrayLocutores, test_size=0.8)

    model_NB.fit(X_train,y_train)

    
    y_hat_NB = model_NB.predict(X_test)
    print(y_hat_NB)
    print(y_test)

    #Utilizaremos y_test para comparar la estimacion con lo que realmente sabemos que hay

    #Calculamos el accuracy
    temp = y_hat_NB == y_test
    

    acc = np.mean(y_hat_NB == y_test)
    print(acc)

#Cálculos para 

/home/patricia/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/patricia/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


['Miguelangel' 'Filippo' 'Adri' 'Patri' 'Filippo' 'Patri' 'Isa' 'Adri'
 'Filippo' 'Miguelangel' 'Filippo' 'Miguelangel' 'Adri' 'Isa' 'Filippo'
 'Patri' 'Miguelangel' 'Patri' 'Miguelangel' 'Adri' 'Patri' 'Isa' 'Isa'
 'Patri' 'Isa']
['Miguelangel', 'Filippo', 'Adri', 'Patri', 'Filippo', 'Patri', 'Isa', 'Adri', 'Filippo', 'Miguelangel', 'Filippo', 'Miguelangel', 'Adri', 'Isa', 'Filippo', 'Patri', 'Miguelangel', 'Patri', 'Miguelangel', 'Adri', 'Patri', 'Isa', 'Isa', 'Patri', 'Isa']
1.0
['Adri' 'Filippo' 'Miguelangel' 'Isa' 'Patri' 'Adri' 'Filippo' 'Adri'
 'Miguelangel' 'Filippo' 'Miguelangel' 'Patri' 'Adri' 'Patri' 'Adri'
 'Miguelangel' 'Filippo' 'Patri' 'Isa' 'Patri' 'Isa' 'Isa' 'Miguelangel'
 'Isa' 'Miguelangel']
['Adri', 'Filippo', 'Miguelangel', 'Isa', 'Patri', 'Adri', 'Filippo', 'Adri', 'Miguelangel', 'Filippo', 'Miguelangel', 'Patri', 'Adri', 'Patri', 'Adri', 'Miguelangel', 'Filippo', 'Patri', 'Isa', 'Patri', 'Isa', 'Isa', 'Miguelangel', 'Isa', 'Miguelangel']
1.0
['Filippo' 'Adri' 'M